In [1]:
import torch
import torch.nn as nn
import torchvision

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import os

from torch.utils.data import Dataset, DataLoader
# from torch.utils.data.sampler import Sampler
import torch.optim as optim
import sys
sys.path.append('../')

from dataset import LbpDataset, train_transforms, val_transforms, test_transforms, collate_fn, get_data
from visualize import visualize
from model import fasterrcnn_resnet201_fpn, FastRCNNPredictor
from engine import evaluate
import utils
from train_lbp import get_train_test_list

In [2]:
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [4]:
import easydict 
args = easydict.EasyDict({ "batch_size": 4, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':5,
                          'workers':4,
                         'output_dir' :'../trained_model/fasterrcnn_resnet50_fpn/'})

In [5]:
df = pd.read_csv('../../data/df.csv')
df.head()
# Data loading code
data_dir = '../../data/df.csv'
train_list, test_list = get_train_test_list(data_dir)
train_dataset = LbpDataset(train_list, transform=train_transforms)
test_dataset = LbpDataset(test_list, transform=val_transforms)  

total 4019 train 3014 test 1005
3014
1005


In [6]:
train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_loader = DataLoader(
    train_dataset, batch_size=args.batch_size,
    sampler=train_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

test_loader = DataLoader(
    test_dataset, batch_size=args.batch_size,
    sampler=test_sampler, num_workers=args.workers,
    collate_fn=utils.collate_fn)

In [7]:
num_classes = 2
model = fasterrcnn_resnet50_fpn(pretrained=True, min_size=2048, max_size=2048)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
device = torch.device('cuda')
model.to(device)
print('model is loaded to gpu')

model is loaded to gpu


In [8]:
# # pretrained.state_dict()
# for p in model.parameters() :
#     if p.requires_grad == True :
#         print(p.shape)

In [9]:
# model.head.regression_head

In [10]:
params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.Adam(params, lr=0.0001)
optimizer = torch.optim.SGD(
       params, lr=0.001, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20, 40, 60, 80, 100], 
                                                    gamma=0.5)

In [ ]:
from train_lbp import train_one_epoch

start_time = time.time()
for epoch in range(120):
    train_one_epoch(model, optimizer, train_loader, device, epoch, 600)
    lr_scheduler.step()
    
    if epoch > 20 and epoch % 5 == 0 :
        if args.output_dir:
            checkpoint = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'lr_scheduler': lr_scheduler.state_dict(),
                'args': args,
                'epoch': epoch
            }
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'model_{}.pth'.format(epoch)))
            utils.save_on_master(
                checkpoint,
                os.path.join(args.output_dir, 'checkpoint.pth'))

        # evaluate after every epoch
        evaluate(model, test_loader, device=device)    
print('total time is {}'.format(time.time() - start_time))    

Epoch: [0]  [  0/754]  eta: 0:51:07  lr: 0.001000  loss: 1.0728 (1.0728)  loss_classifier: 0.8199 (0.8199)  loss_box_reg: 0.0059 (0.0059)  loss_objectness: 0.2283 (0.2283)  loss_rpn_box_reg: 0.0186 (0.0186)  time: 4.0687  data: 1.9854  max mem: 18498
Epoch: [0]  [600/754]  eta: 0:03:05  lr: 0.001000  loss: 0.0771 (0.1084)  loss_classifier: 0.0244 (0.0285)  loss_box_reg: 0.0084 (0.0086)  loss_objectness: 0.0284 (0.0456)  loss_rpn_box_reg: 0.0056 (0.0256)  time: 1.2021  data: 0.0385  max mem: 18498
Epoch: [0]  [753/754]  eta: 0:00:01  lr: 0.001000  loss: 0.0961 (0.1078)  loss_classifier: 0.0336 (0.0292)  loss_box_reg: 0.0137 (0.0091)  loss_objectness: 0.0285 (0.0453)  loss_rpn_box_reg: 0.0053 (0.0242)  time: 1.1689  data: 0.0354  max mem: 18498
Epoch: [0] Total time: 0:15:05 (1.2005 s / it)
Epoch: [1]  [  0/754]  eta: 0:40:16  lr: 0.001000  loss: 0.0771 (0.0771)  loss_classifier: 0.0212 (0.0212)  loss_box_reg: 0.0025 (0.0025)  loss_objectness: 0.0452 (0.0452)  loss_rpn_box_reg: 0.0083 (0

In [ ]:
evaluate(model, test_loader, device=device) 